In [1]:
from Class_PINN_FBPINN import *
import matplotlib.pyplot as plt
import pickle

Running on  mps


### Increasing number of subdomains and layers in PINN for multi-scale problem

In [2]:
# Parameters
domain = [-2*torch.pi, 2*torch.pi]

n_multi_scale = 5
w_list = [2, 4, 8, 16, 32]
overlap = 0.3
sigma = 0.1
n_hidden_layers = 2
neurons = 16

#n_subdomains = 30

num_points = 200*15
# n_epochs = 10000
n_epochs = 5000

n_subdomains = [10,30,50,100]
# n_subdomains = [10, 30, 50, 100]

In [3]:
l1_loss_arr = []
l1_loss_min = []
loss_domains = []

In [4]:
def plot_solution():
    # Plot the exact solution
    x = torch.linspace(domain[0], domain[1], num_points, dtype=torch.float32, device=DEVICE).reshape(-1, 1)
    u_exact = model.exact_solution(x)

    plt.plot(x.cpu().detach().numpy(), u_exact.cpu().detach().numpy(), label='Exact', color="blue")

    # Plot the FBPINN solution
    for i in range(model.n_subdomains):
        subdomain_i = model.subdomains[i]
        NN_i = model.neural_networks[i]
        NN_i.eval()

    u_approx = torch.tanh(model.w_list[-1] * x) * model(x)

    plt.plot(x.cpu().detach().numpy(), u_approx.cpu().detach().numpy(), color="green")
    # Add label only once
    if i == 0:
        plt.plot(x.cpu().detach().numpy(), u_approx.cpu().detach().numpy(), label='FBPINN', color="green")
    plt.title('FBPINN (full solution)')
    plt.legend()
    plt.show()

In [5]:
mps_device = torch.device("mps")

for nr_sub in n_subdomains:

    # Create the model
    model = FBPINN_Cos_nD(domain_extrema=domain, n_subdomains=nr_sub, overlap=overlap, sigma=sigma, n_hidden_layers=n_hidden_layers, neurons=neurons, activation_function=nn.Tanh(), n_multi_scale=n_multi_scale, w_list=w_list)

    # Train the FBPINN
    model.to(mps_device)
    history, l1_loss = model.fit(num_points, n_epochs, verbose=False)

    l1_loss_arr.append(l1_loss)

    print("Loss: {}".format(np.min(l1_loss)))

    fig = plt.figure(figsize=(10, 5))

    plot_solution()

    torch.save(model.state_dict(), 'models_save/scale_fbpinn_{}.pdh'.format(nr_sub))

    # Save the l1 loss in pickle
    with open("l1_loss_scale_fbpinn_{}".format(nr_sub), 'wb') as file:
        pickle.dump(l1_loss, file)

AttributeError: 'Tanh' object has no attribute 'device'

In [ ]:
# Parameters
domain = [-2*torch.pi, 2*torch.pi]

n_multi_scale = 3
w_list = [1, 4, 15]
n_subdomains = 30
overlap = 0.3
sigma = 0.1
n_hidden_layers = 2
neurons = 16

# Create the model
fbpinn = FBPINN_Cos_nD(domain_extrema=domain, n_subdomains=n_subdomains, overlap=overlap, sigma=sigma, n_hidden_layers=n_hidden_layers, neurons=neurons, activation_function=nn.Tanh(), n_multi_scale=n_multi_scale, w_list=w_list)

# Train the FBPINN
num_points = 200*15
n_epochs = 10000

history = fbpinn.fit(num_points, n_epochs, verbose=False)